In [5]:
cc.VerificationHandler.close_browser()

In [308]:
VerificationHandler.close_browser()

In [132]:
import numpy as np
import pandas as pd
from bs4.element import NavigableString

In [2]:
% run contactsScraper.py

Done
Done
KEYS
Done
Done
RECORDS COLLECTED
DATAFRAMES READY


In [3]:
% run contactChecker.py

Local Contact Checker Ready


In [4]:
ContactSheetOutput.set_output(contactKeys)
VerificationHandler.set_orgRecords(dm.OrgSession(orgRecords))
VerificationHandler.set_contactRecords(cr)
print('Local Contact Checker Ready')

RECORD OUTPUT READY
Local Contact Checker Ready


In [5]:
orgsForToday = ['National Association for Multi-Ethnicity In Communications (NAMIC)',
                'Association for Women in Science',
                'Brain Injury Association of America',
                'American Society of Home Inspectors',
                'NAADAC, the Association for Addiction Professionals',
                'American Public Transportation Association',
                'Indiana Soybean Alliance',
                'Associated Builders and Contractors (ABC)',
                'National Association of Social Workers',
                'American Marketing Association (AMA)']

In [261]:
org  = orgsForToday[0]

vh = MotherSetVerifier(org)

In [101]:
org  = orgsForToday[0]

vh = ContactScraperVerifier(org)

In [102]:
VerificationHandler.orgRecords.orgSessionStatusCheck()

['Mon Oct 23, 2017  19:29:39', '2.776', 's', 'Good']

In [105]:
ContactSheetOutput.currentRow

258

In [296]:
vh.write_contact_pointers()

In [106]:
class ContactScraperVerifier(MotherSetVerifier):
    def __init__(self, org):
        MotherSetVerifier.__init__(self, org)
        self.pointers = self.verifiedPointers
        
        # Get Grand Mother elements
        self.gmElements, self.grandMotherMatrix = ContactScraperVerifier.getGrandMotherElements(self.pointers)
        
        self.noGm = len(self.gmElements)
        
        if self.noGm == 1: ## Single Grandmother Case
            self.gm = self.gmElements[0]
            self.distinct_gm = ContactScraperVerifier.distinct_gm(self.gm, self.pointers)
            
            #if ContactScraperVerifier.distinct_gm(self.gm, self.pointers): ## Grandmother is distinct
            ### Extender Model Selection
            #    self.extenders = [Extender(self.gm, vp) for vp in self.pointers]
            #
            #else:  ## GrandMother is not distinct
            #    ## Extender Model Selection
            #    self.extenders = [RocketOnlyExtender(self.gm, vp) for vp in self.pointers]
            
        else:
            ## Extender Model Selection for Multiple GrandMothers
            self.extenders = None
            self.gm = None
            
    @staticmethod
    def getGrandMotherElements(pointers):
        ## Identify Grandmother elements
        gmElements = []
        gmMatrix = []

        for i in range(len(pointers)):
            igmElements = []
            for j in range(i):
                ## Check to see if the Any Mother element is a Big Momma or "Bertha" Element
                if pointers[i].get_mother_element() is pointers[j].get_mother_element():
                    gm = pointers[i].get_mother_element()
                else:
                    gm = pointers[i].common_parent(pointers[j])
                # Append Match to Grand Mother Matrix
                igmElements.append(gm)

                # Check to see if this is a new grand mother element,
                # if so append to the gmElements list of unique grandmother elements 
                if gm not in gmElements:
                    gmElements.append(gm)

            # Append Matrix Row
            gmMatrix.append(igmElements)

        grandMotherMatrix = np.matrix(gmMatrix)
        return (gmElements, grandMotherMatrix)
    
    @staticmethod
    def distinct_gm(gm, pts):
        if len(pts) == 0:
            return True
        if gm is pts[0].get_mother_element():
            return False
        else:
            return ContactScraperVerifier.distinct_gm(gm, pts[1:])

In [832]:
class Extender(object):
    def __init__(self, gm, pointer):
        self.gm = gm
        self.vp = pointer
        
        self.tag_nathans()
        self.tag_toms()
        
        self.tom_missile = self.mother_to_tom_missile()
        self.tom_rocket = self.nathan_to_tom_rocket()
        self.nathan_shuttle = self.nathan_to_element()
        
        self.nathan_missile = self.mother_to_nathan_missile()
        self.nathan_rocket = self.tom_to_nathan_rocket()
        self.tom_shuttle = self.tom_to_element()
        
        #self.reset_tree()
        
    ## Tagging Functions ------------------------------------ 
    ##
    def tag_nathans(self):
        if self.vp.get_mother_element() is self.vp.nathan.parent:  ## Gotta Catch those first elements before the get away
            self.vp.nathan.parent['nathan'] = 0
        else:
            return Extender.parent_cycle_up(self.vp.get_mother_element(), self.vp.nathan.parent, 'nathan', 0)

    def tag_toms(self):
        if self.vp.get_mother_element() is self.vp.tom.parent:  ## Gotta Catch those first elements before the get away
            self.vp.tom.parent['tom'] = 0
        else:
            return Extender.parent_cycle_up(self.vp.get_mother_element(), self.vp.tom.parent, 'tom', 0)
    
    def reset_tree(self):
        ## Clear all tom, nathan and sib atributes
        for tomTag in self.gm.find_all(Extender.has_tom):
            del tomTag['tom']    
        for nathanTag in self.gm.find_all(Extender.has_nathan):
            del nathanTag['nathan'] 
        for sibTag in self.gm.find_all(Extender.has_sib):
            del sibTag['sib']
    
    @staticmethod
    def parent_cycle_up(motherElement, element, atr, num):
        ## We tag up THOUGH the MotherELement
        if element is motherElement:
            element[atr] = num
            #element['sib'] = atr
        else:
            element[atr] = num
            return Extender.parent_cycle_up(motherElement, element.parent, atr, num + 1)
    
    @staticmethod
    def contents_position(elm):
        return Extender.contents_position_loop(elm, 0)

    @staticmethod
    def contents_position_loop(elm, num):
        if len(list(elm.previous_siblings)) == 0:
            return num
        else:
            return Extender.contents_position_loop(elm.previous_sibling, num + 1)
    
    @staticmethod
    def has_tom(tag):
        try:
            return 'tom' in tag.attrs
        except AttributeError:
            return False

    @staticmethod
    def has_nathan(tag):
        try:
            return 'nathan' in tag.attrs
        except AttributeError:
            return False

    @staticmethod
    def has_sib(tag):
        try:
            return 'sib' in tag.attrs
        except AttributeError:
            return False
        
    @staticmethod
    def check_siblings(sibs, attFunc):
        for sib in sibs:
            if attFunc(sib):
                return True
        return False
    
    ## Shuttle Functions ------------------------------------
    ##
    def nathan_to_element(self):
        return lambda start: start.contents[Extender.contents_position(self.vp.nathan)]
    
    def tom_to_element(self):
        return lambda start: start.contents[Extender.contents_position(self.vp.tom)]
    
    ## Missle Functions ------------------------------------
    ##
    def mother_to_tom_missile(self):
        motherElement = self.vp.get_mother_element()

        if self.vp.tom is motherElement:
            return lambda start: start
        else:
            return lambda start: Extender.cycle_up_mother(self.vp.tom.parent, motherElement, start).contents[Extender.contents_position(self.vp.tom)]

    def mother_to_nathan_missile(self):
        motherElement = self.vp.get_mother_element()

        if self.vp.nathan is motherElement:
            return lambda start: start
        else:
            return lambda start: Extender.cycle_up_mother(self.vp.nathan.parent, motherElement, start).contents[Extender.contents_position(self.vp.nathan)]

    @staticmethod
    def cycle_up_mother(elm, motherElement, start):
        if elm is motherElement:
            return start
        else:
            return Extender.cycle_up_mother(elm.parent, motherElement, start).contents[Extender.contents_position(elm)]
        
    
    ## Rocket Functions ------------------------------------
    ##
    def nathan_to_tom_rocket(self):
        ## Compute the route from nathan startnode to tom by starting at tom and recursing up, accross and then down
        ## looing for nathan = 0 attribute
        
        return lambda start: Extender.cycle_up(self.vp.tom, 'nathan', start)

    def tom_to_nathan_rocket(self):
        ## Compute the route from tom startnode to nathan by starting at nathan and recursing up, accross and then down
        ## looing for tom = 0 attibute
        
        return lambda start: Extender.cycle_up(self.vp.nathan, 'tom', start)
    
    @staticmethod
    def cycle_up(elm, to, start):
        ## Set Attribut Search Function
        attrFunc = Extender.has_tom if to == 'tom' else Extender.has_nathan

        ## FIRST check yo'self
        if attrFunc(elm):  ## Switch Direction but don't move
            return Extender.cycle_down(elm, to, start) 
        ## SECOND look Left for shoulder in siblings
        elif Extender.check_siblings(elm.previous_siblings, attrFunc):
            return Extender.cycle_left(elm.previous_sibling, to, start).next_sibling
        ## Then look right
        elif Extender.check_siblings(elm.next_siblings, attrFunc):
            return Extender.cycle_right(elm.next_sibling, to, start).previous_sibling
        ## if nothing here go up
        else:
            return Extender.cycle_up(elm.parent, to, start).contents[Extender.contents_position(elm)]

    @staticmethod
    def cycle_left(elm, to, start):
        attrFunc = Extender.has_tom if to == 'tom' else Extender.has_nathan

        ## if this is the shoulder change functions but dont move
        if attrFunc(elm):
            return Extender.cycle_down(elm, to, start)
        ## if not check the element to the left
        else:
            return Extender.cycle_left(elm.previous_sibling, to, start).next_sibling

    @staticmethod
    def cycle_right(elm, to, start):
        attrFunc = Extender.has_tom if to == 'tom' else Extender.has_nathan

        ## if this is the shoulder change functions but dont move
        if attrFunc(elm):
            return Extender.cycle_down(elm, to, start)
        ## if not check the element to the left
        else:
            return Extender.cycle_right(elm.next_sibling, to, start).previous_sibling

    @staticmethod
    def cycle_down(elm, to, start):
        return Extender.cycle_down_loop(elm[to], start)

    @staticmethod
    def cycle_down_loop(dist, start):
        ## Start Node Condition
        if dist == 0:
            return start
        else: 
            return Extender.cycle_down_loop(dist - 1, start).parent
 
        

In [833]:
class Extractor(Extender):
    nathanStartType = '__nathanStarts__'
    tomStartType = '__tomStarts__'
    motherStartType = '__motherStarts__'
    
    def __init__(self, gm, pointer):
        Extender.__init__(self, gm, pointer)
        self.startBlock = StartBlockNoMother(gm, pointer)
        self.startsType, self.starts = self.get_starts()
        self.resultSet = NewPointerSet()
        self.test_starts()
        self.reset_tree()
        
    def get_starts(self):
        return self.startBlock.get_optimal_starts()
    
    def test_starts(self):
        return [self.test_start(self.startsType, st) for st in self.starts]
        
    def test_start(self, startType, start):
        if startType == Extractor.nathanStartType:   ## Nathan Start Case, Nathan Shuttle, Tom Rocket to 
            np = NewPointer(start, self.nathan_shuttle, self.tom_rocket)
            self.resultSet.addPointer(np)
            return np
        if startType == Extractor.tomStartType:   ## Tom Start Case, Tom Shuttle, Nathan Rocket 
            np = NewPointer(start, self.nathan_rocket, self.tom_shuttle)
            self.resultSet.addPointer(np)
            return np
        if startType == Extractor.motherStartType:   ## Mother Start Case, Missile for Missle for Nathan, Missile for Tom 
            np = NewPointer(start, self.nathan_missile, self.tom_missile)
            self.resultSet.addPointer(np)
            return np
    
    def get_result_set(self):
        return self.resultSet
    
    def quick_report(self):
        print(str(self.resultSet))
        

In [834]:
class ExtractorNoMother(Extractor):
    nathanStartType = '__nathanStarts__'
    tomStartType = '__tomStarts__'
    motherStartType = '__motherStarts__'
    
    def __init__(self, gm, pointer):
        Extractor.__init__(self, gm, pointer)
        self.startBlock = StartBlockNoMother(gm, pointer)
        self.startsType, self.starts = self.get_starts()
        self.reset_tree()
        
    def get_starts(self):
        return self.startBlock.get_optimal_starts()

In [835]:
class NewPointerSet(object):
    def __init__(self):
        self.newPointers = []
        
    def addPointer(self, newPointer):
        self.newPointers.append(newPointer)
        
    def __str__(self):
        s = ''
        for pt in self.newPointers:
            s += ('%s     as     %s\n' % (str(pt.get_nathan()), str(pt.get_tom())))
        return s
    
    def get_pointers(self):
        return self.newPointers
    

In [836]:
class MergeSet(NewPointerSet):
    def __init__(self, vps):
        NewPointerSet.__init__(self)
        self.verifiedPointers = vps
        
    def merge_pointers(self, newPointerSet):
        for np in newPointerSet.get_pointers():
            self.add_pointer(np)
    
    def add_pointer(self, newPointer):
        ## Pointer has to meet certain conditions to be added to the MergeSet
        if MergeSet.has_a_failed_route(newPointer):
            return False
        if MergeSet.is_nothing(newPointer.tom) or MergeSet.is_nothing(newPointer.nathan):
            return False
        if MergeSet.is_an_email(newPointer.tom) or MergeSet.is_an_email(newPointer.nathan):
            return False
        if self.a_match_to_verified_pointers(newPointer):
            return False
        if self.a_match_in_set(newPointer):
            return False
        
        ## Add to MergeSetCase
        self.newPointers.append(newPointer)
                                                    
    @staticmethod
    def is_an_email(pointerWord):
        if '@' in pointerWord:
            return True
        return False
    
    @staticmethod                                                   
    def is_nothing(pointerWord):
        letters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'       
        for char in pointerWord:
            if char in letters:
                return False
        return True 
    
    @staticmethod
    def has_a_failed_route(pointer):
        if not pointer.tom:
            return True
        if not pointer.nathan:
            return True
        return False
    
    def a_match_in_set(self, pointer):
        for np in self.newPointers:
            if pointer.nathan is np.nathan:
                return True
            if pointer.tom is np.tom:
                return True
        return False
    
    def a_match_to_verified_pointers(self, pointer):
        for vp in self.verifiedPointers:
            if pointer.nathan is vp.nathan:
                return True
            if pointer.tom is vp.tom:
                return True
        return False
        

In [837]:
class NewPointer(object):
    def __init__(self, start, nathanTest, tomTest):
        self.nathanRoute = nathanTest
        self.tomRoute = tomTest
        self.start = start
        
        ## Execute Test
        self.nathan = self.test_nathan()
        self.tom = self.test_tom()
        
        self.output = ContactSheetOutput('New Pointer For: %s start' % str(start))
        
    def test_nathan(self):
        try:
            result = self.nathanRoute(self.start)
            if type(result) is not NavigableString:
                result = None
        except:
            result = None
            
        return result
    
    def test_tom(self):
        try:
            result = self.tomRoute(self.start)
            if type(result) is not NavigableString:
                result = None
        except:
            result = None
            
        return result
    
    def get_tom(self):
        return self.tom
    
    def get_nathan(self):
        return self.nathan

In [838]:
class StartBlock(object):
    def __init__(self, gm, pointer):
        self.vp = pointer
        self.gm = gm
        
        self.nathanNameStarts = gm.find_all(pointer.nathan.parent.name)
        self.noNathanNameStarts = len(self.nathanNameStarts)
        self.tomNameStarts = gm.find_all(pointer.tom.parent.name)
        self.noTomNameStarts = len(self.tomNameStarts)
        self.motherNameStarts = gm.find_all(pointer.get_mother_element().name)
        self.noMotherNameStarts = len(self.motherNameStarts)
        
        self.noNathanClasses = self.get_no_nathan_classes()
        self.noTomClasses = self.get_no_tom_classes()
        self.noMotherClasses = self.get_no_mother_classes()
        
    def get_no_nathan_classes(self):
        if 'class' in self.vp.nathan.parent.attrs:
            return len(self.vp.nathan.parent['class'])
        else:
            return 0
    
    def get_no_tom_classes(self):
        if 'class' in self.vp.tom.parent.attrs:
            return len(self.vp.tom.parent['class'])
        else:
            return 0        
    
    def get_no_mother_classes(self):
        if 'class' in self.vp.get_mother_element().attrs:
            return len(self.vp.get_mother_element()['class'])
        else:
            return 0
        
    def get_nathan_class_starts(self):
        return self.gm.find_all(class_=self.vp.nathan.parent['class'])
    
    def get_tom_class_starts(self):
        return self.gm.find_all(class_=self.vp.tom.parent['class'])
    
    def get_mother_class_starts(self):
        return self.gm.find_all(class_=self.vp.get_mother_element()['class'])
    
    def get_optimal_starts(self):
        maxNoClasses = max([self.noNathanClasses, self.noTomClasses, self.noMotherClasses])
        minNoNameStarts = min([self.noNathanNameStarts, self.noTomNameStarts, self.noMotherNameStarts])
        
        #dispatch
        if maxNoClasses > 0: ## There are classes to choose better starts
            if self.noMotherClasses == maxNoClasses:
                return '__motherStarts__', self.get_mother_class_starts()
            elif self.noNathanClasses == maxNoClasses:
                return '__nathanStarts__', self.get_nathan_class_starts()
            else:
                return '__tomStarts__', self.get_tom_class_starts()
        else:  ## There are no classes to test mus go by the minimun number of name matches
            if self.noMotherNameStarts == minNoNameStarts:
                return '__motherStarts__', self.motherNameStarts
            elif self.noNathanNameStarts == minNoNameStarts:
                return '__nathanStarts__', self.nathanNameStarts
            else:
                return '__tomStarts__', self.tomNameStarts       

In [839]:
class StartBlockNoMother(StartBlock):
    def __init__(self, gm, pointer):
        StartBlock.__init__(self, gm, pointer)
    
    def get_optimal_starts(self):
        ## Unlike get_optimal_starts from the inheritted StartBlock this one does not return mother starts
        maxNoClasses = max([self.noNathanClasses, self.noTomClasses])
        minNoNameStarts = min([self.noNathanNameStarts, self.noTomNameStarts])
        
        #dispatch
        if maxNoClasses > 0: ## There are classes to choose better starts
            if self.noNathanClasses == maxNoClasses:
                return '__nathanStarts__', self.get_nathan_class_starts()
            else:
                return '__tomStarts__', self.get_tom_class_starts()
        else:  ## There are no classes to test mus go by the minimun number of name matches
            if self.noNathanNameStarts == minNoNameStarts:
                return '__nathanStarts__', self.nathanNameStarts
            else:
                return '__tomStarts__', self.tomNameStarts

In [840]:
class Dorito(object):
    def __init__(self, gm, pointers):
        self.grandMotherElement = gm
        self.verifiedPointers = pointers
        self.scrapePointers = Dorito.filterPointers(pointers)
        self.result = None
        self.finalPointers = MergeSet(pointers)
    
    def extract(self):
        ## This class will test the first One
        try:
            self.x = Extractor(self.grandMotherElement, self.scrapePointers[0])
            self.result = self.x.get_result_set()
        except IndexError:
            print("No Scrapable Matches")
        
    def merge_result(self):
        if self.result:
            self.finalPointers.merge_pointers(self.result)
            if not str(self.finalPointers):
                print("Nothing passed Merge")
        else:
            print("Nothing to Merge")    
        self.report()
            
    def report(self):
        print('Verified Pointers         \t\t\t\t\t\t%s' % len(self.verifiedPointers))
        print('Scrape Pointers           \t\t\t\t\t\t%s' % len(self.scrapePointers))
        print('Start Pointers            \t\t\t\t\t\t%s' % len(self.result.get_pointers()) if self.result else 0)
        print('Start Type                \t\t\t\t%s' % self.x.startsType)
        print('Merged (Filtered) Pointers\t\t\t\t\t\t%s' % len(self.finalPointers.get_pointers()))
        print(self.finalPointers)
    
    @staticmethod
    def filterPointers(vps):
        return [pointer for pointer in vps if pointer.mary_here() and pointer.nathan_here]

In [ ]:
org  = orgsForToday[2]

vh = ContactScraperVerifier(org)

In [846]:
d = Dorito(vh.gm, vh.pointers)

In [825]:
ds = Dorito(vh.pointers[0].get_mother_element().parent, vh.pointers)
ds.extract()
ds.merge_result()

Nothing passed Merge
Verified Pointers         						4
Scrape Pointers           						4
Start Pointers            						10
Start Type                				__tomStarts__
Merged (Filtered) Pointers						0



In [847]:
d.extract()

In [848]:
d.merge_result()

Verified Pointers         						2
Scrape Pointers           						2
Start Pointers            						86
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						81
Rosemary Sheridan     as     Vice President - Communications & Marketing​
Mantill Williams     as     Director - Advocacy Communications
Chad Chitwood     as     Program Manager - Advocacy Communications
Virginia Miller     as     Director - Media Relations
Kathy Golden     as     Senior Director - Publications & Passenger Transport
Vacant     as     Senior Managing Editor - Communications
Susan Berlin     as     Senior Editor
Bill Maroni     as     Senior Strategist, Executive Communications
Mitch Wood     as     Graphic Designer
Jack Gonzalez     as     Senior Director - Marketing & Sales
Stephen Kendrick     as     Program Manager - Communications & Marketing
Erin Cartwright     as     Communications & Marketing Coordinator
Daniel Pineda     as     Video Production Specialist
Vacant     as     Senior 

In [695]:
"@" in d.finalPointers.get_pointers()[3].nathan 

True

In [731]:
len(d.finalPointers.get_pointers())

81

In [744]:
print(d.result)

None     as                  President/CEO 
None     as                  Development Manager
None     as     None
None     as                  Public Affairs Manager
None     as                  ACBIS Assistant
None     as                  Marketing and Communications Coordinator 
None     as                  BIAA - Maine Program Coordinator
None     as                  Sales Manager
None     as                  Executive Vice President/COO 
None     as                  BIAA-Maine Executive Director



In [811]:
st = d.x.starts

In [812]:
len(st)

10

In [813]:
st

[<p><a href="mailto:sconnors@biausa.org">Susan Connors </a><br/>             President/CEO <br/>             703-761-0750 <br/> <br/> <a href="mailto:mabashian@biausa.org">Marianna Abashian</a><br/>             Director of Professional Services <br/>             703-761-0750 x636 <br/> <br/> <a href="mailto:gayotte@biausa.org">Gregory Ayotte</a><br/>             Director of Consumer Services <br/>             703-761-0750 x642 <br/> <br/> <a href="mailto:acolberg@biausa.org">Amy C. Colberg</a><br/>             Director of Government Affairs <br/>             703-761-0750 x637</p>,
 <p><a href="mailto:scohen@biausa.org?subject=Website inquiry">Stephanie Cohen</a><br/>             Development Manager<br/>             703-761-0750 x645 </p>,
 <p> </p>,
 <p><a href="mailto:wdane@biausa.org?subject=inquiry from Contact Us Page BIAA website">William M. Dane</a><br/>             Public Affairs Manager<br/>             703-761-0750 x644<br/>             512-987-0101</p>,
 <p><a href="mailto:ac

In [554]:
scrapePointers = Dorito.filterPointers(vh.pointers)

In [567]:
x = ExtractorNoMother(vh.gm, scrapePointers[0])

In [564]:
## No Grand Mother Single Pointer Version
y = ExtractorNoMother(vh.pointers[0].get_mother_element().parent, vh.pointers[0])

In [565]:
y.quick_report()

None     as                  President/CEO 
None     as                  Development Manager
None     as     None
None     as                  Public Affairs Manager
None     as                  ACBIS Assistant
None     as                  Marketing and Communications Coordinator 
None     as                  BIAA - Maine Program Coordinator
None     as                  Sales Manager
None     as                  Executive Vice President/COO 
None     as                  BIAA-Maine Executive Director



In [477]:
print(x.resultSet)

Russ Klein​​     as     Chief Executive Officer - 
Andy Friedman​     as     Chief Content Officer - 
Barbara Grobicki​​     as     Chief Alliance Officer - 
Jennifer Severns​​     as     Chief Experience Officer - 
Jeremy Van Ek​​     as     Chief Operations Officer - 
Michael Pemberton​     as     Accounting - 
None     as     None
Adara Bowen     as     None
None     as     None
None     as     None
Matt Weingarden​     as      
info@ama.org​     as     For membership inquiries, please email us at 



In [456]:
vh.pointers[0].nathan_here()

True

In [457]:
len(x.newPointers)

AttributeError: 'ExtractorNoMother' object has no attribute 'newPointers'

In [414]:
vh.write_contact_pointers()

In [561]:
len(vh.pointers)

4

In [570]:
vh.pointers[1].get_mother_element()

<p><a href="mailto:mreitter@biausa.org">Mary S. Reitter, CAE</a><br/>             Executive Vice President/COO <br/>             703-761-0750 x623</p>

In [563]:
vh.noGm

1

In [581]:
vh.pointers[0].tom

'             President/CEO '

In [582]:
vh.pointers[0].nathan

'Susan Connors '

In [583]:
vh.pointers[0].get_mother_element() is vh.gm

False

In [584]:
vh.distinct_gm

True

In [585]:
sb = StartBlockNoMother(vh.gm, vh.pointers[0])

In [588]:
sb.noNameStarts

14

In [577]:
sb.get_optimal_starts()

('__tomStarts__',
 [<p><a href="mailto:sconnors@biausa.org">Susan Connors </a><br/>             President/CEO <br/>             703-761-0750 <br/> <br/> <a href="mailto:mabashian@biausa.org">Marianna Abashian</a><br/>             Director of Professional Services <br/>             703-761-0750 x636 <br/> <br/> <a href="mailto:gayotte@biausa.org">Gregory Ayotte</a><br/>             Director of Consumer Services <br/>             703-761-0750 x642 <br/> <br/> <a href="mailto:acolberg@biausa.org">Amy C. Colberg</a><br/>             Director of Government Affairs <br/>             703-761-0750 x637</p>,
  <p><a href="mailto:scohen@biausa.org?subject=Website inquiry">Stephanie Cohen</a><br/>             Development Manager<br/>             703-761-0750 x645 </p>,
  <p> </p>,
  <p><a href="mailto:wdane@biausa.org?subject=inquiry from Contact Us Page BIAA website">William M. Dane</a><br/>             Public Affairs Manager<br/>             703-761-0750 x644<br/>             512-987-0101</p>,


In [576]:
a2 = sb.get_optimal_starts()[1]

In [602]:
vh.pointers[1].tom

'             Executive Vice President/COO '

In [601]:
a2

[<p tom="0"><a href="mailto:sconnors@biausa.org" nathan="0">Susan Connors </a><br/>             President/CEO <br/>             703-761-0750 <br/> <br/> <a href="mailto:mabashian@biausa.org">Marianna Abashian</a><br/>             Director of Professional Services <br/>             703-761-0750 x636 <br/> <br/> <a href="mailto:gayotte@biausa.org">Gregory Ayotte</a><br/>             Director of Consumer Services <br/>             703-761-0750 x642 <br/> <br/> <a href="mailto:acolberg@biausa.org">Amy C. Colberg</a><br/>             Director of Government Affairs <br/>             703-761-0750 x637</p>,
 <p><a href="mailto:scohen@biausa.org?subject=Website inquiry">Stephanie Cohen</a><br/>             Development Manager<br/>             703-761-0750 x645 </p>,
 <p> </p>,
 <p><a href="mailto:wdane@biausa.org?subject=inquiry from Contact Us Page BIAA website">William M. Dane</a><br/>             Public Affairs Manager<br/>             703-761-0750 x644<br/>             512-987-0101</p>,
 <p

In [236]:
sb.get_tom_class_starts()

[<h2 class="h2_addpading"> President </h2>,
 <h2 class="h2_addpading"> Vice President </h2>,
 <h2 class="h2_addpading"> Secretary </h2>,
 <h2 class="h2_addpading"> Treasurer </h2>]

In [271]:
a = sb.get_nathan_class_starts()

In [598]:
testEx = Extender(vh.gm, vh.pointers[0])

In [599]:
n = 4
testEx.tom_shuttle(a2[n])

'             ACBIS Assistant'

In [600]:
testEx.nathan_rocket(a2[n])

AttributeError: 'NoneType' object has no attribute 'previous_siblings'

In [253]:
list(vh.pointers[0].tom.next_siblings)

[]

In [216]:
x.starts

[<h3 class="h3_addpading"> Mildred (Mit) Joyner, MSW, LCSW, BCD </h3>,
 <h3 class="h3_addpading">Heather Bradley-Geary, MSW</h3>,
 <h3 class="h3_addpading">Patrick J. Johnson, LCSW</h3>]

In [217]:
x.startsType

'__nathanStarts__'

In [406]:
testEx.reset_tree()

In [405]:
len(a2)

19

In [158]:
x.get_new_pointers()

In [163]:
for pt in x.get_new_pointers():
    print('%s     as     %s' % (str(pt.get_nathan()), str(pt.get_tom())))

Eglon E. Simons     as     President and CEO
James C. Jones     as     Senior Vice President, Education and Diversity Solutions
Sandra Girado     as     Vice President, Events and Partner Relations
S. Monique Wells     as     Director of Finance and Administration
Susan Waldman     as     Senior Manager, Membership and Publications
Darryl Jackson     as     Manager, Mentoring Relations and Chapter Operations
Brandon D. Hewlett     as     Project Coordinator
Davana O'Brien     as     Administrative Assistant


In [111]:
ts = StartBlock(vh.gm, vh.pointers[0])

In [112]:
ts.get_optimal_starts()

('__motherStarts__',
 [<div class="team-member-right"><h3 class="team-member-name">Eglon E. Simons</h3><h4 class="team-member-position">President and CEO</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">Eglon E. Simons leads the association and champions its mission as President and CEO. In addition to working closely with NAMIC’s 16 chapters across the country, he collaborates with NAMIC’s board, staff, chapter leadership and industry partners in building upon NAMIC’s legacy. A strong and progressive leader, Simons’ career in media and communications has spanned more than 30 years. Upon earning an MBA from Harvard Business School, Simons joined the staff at CBS New York. After holding several managerial positions, Simons concluded his 13-year run with the network as Vice President of Sales and Marketing for the CBS Television Stations Division. In 1990, Simons embarked on a 21-year run with Cablevision Systems Corporation as Director of New Yor

In [85]:
len(ts.get_optimal_starts()[1])

8

In [113]:
ns = StartBlockNoMother(vh.gm, vh.pointers[0])

In [114]:
ns.get_optimal_starts()

('__nathanStarts__',
 [<h3 class="team-member-name">Eglon E. Simons</h3>,
  <h3 class="team-member-name">James C. Jones</h3>,
  <h3 class="team-member-name">Sandra Girado</h3>,
  <h3 class="team-member-name">S. Monique Wells</h3>,
  <h3 class="team-member-name">Susan Waldman</h3>,
  <h3 class="team-member-name">Darryl Jackson</h3>,
  <h3 class="team-member-name">Brandon D. Hewlett</h3>,
  <h3 class="team-member-name">Davana O'Brien</h3>])

In [89]:
len(ns.get_optimal_starts()[1])

8

In [93]:
vh.extenders[0].reset_tree()

In [120]:
s = ns.get_optimal_starts()[1][0]

In [122]:
str(s)

'<h3 class="team-member-name">Eglon E. Simons</h3>'

In [133]:
type(vh.pointers[0].tom) is not NavigableString

False

In [129]:
from bs4.element import NavigableString

In [68]:
min([1,2,3])

1

In [23]:
vp = vh.pointers[0]

In [24]:
vp.tom

'Senior Manager, Membership and Publications'

In [25]:
vp.nathan

'Susan Waldman'

In [26]:
vp.nathan.parent.name

'h3'

In [27]:
vp.nathan.parent['class']

['team-member-name']

In [28]:
vp.nathan.parent.attrs

{'class': ['team-member-name'], 'sib': 'nathan'}

In [43]:
vh.extenders[2].reset_tree()

In [67]:
len(vh.gm.find_all(vp.get_mother_element().name))

64

In [31]:
gm.find_all(class_=vp.nathan.parent['class'][0])

NameError: name 'gm' is not defined

In [ ]:
gm.find_all

In [13]:
## Identify Grandmother elements
pointers = vh.verifiedPointers
gmElements = []
gmMatrix = []

for i in range(len(pointers)):
    igmElements = []
    for j in range(i):
        ## Check to see if the Any Mother element is a Big Momma or "Bertha" Element
        if pointers[i].get_mother_element() is pointers[j].get_mother_element():
            gm = pointers[i].get_mother_element()
        else:
            gm = pointers[i].common_parent(pointers[j])
        # Append Match to Grand Mother Matrix
        igmElements.append(gm)
        
        # Check to see if this is a new grand mother element,
        # if so append to the gmElements list of unique grandmother elements 
        if gm not in gmElements:
            gmElements.append(gm)
            
    # Append Matrix Row
    gmMatrix.append(igmElements)
    
grandMotherMatrix = np.matrix(gmMatrix)

In [14]:
noGm = len(gmElements)

In [15]:
noGm

1

In [21]:
## if 1 grandmother element test set it and test it
gm = gmElements[0]

In [29]:
def distinct_gm(gm, pts):
    if len(pts) == 0:
        return True
    if gm is pts[0].get_mother_element():
        return False
    else:
        return distinct_gm(gm, pts[1:])

In [31]:
## check to see if Grandmother is Distinct
distinct = distinct_gm(gm, pointers)

In [32]:
distinct

True

In [25]:
pointers[1].get_mother_element()

<div class="team-member-right"><h3 class="team-member-name">S. Monique Wells</h3><h4 class="team-member-position">Director of Finance and Administration</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">Monique Wells is the Director of Finance and Administration for the National Association for Multi-ethnicity in Communications’ (NAMIC). Prior to that she was their Manager of Operations, a position she assumed in August, 2009. Monique is no stranger to NAMIC and has been a member since 1991. She served as a consultant to NAMIC for a number of special projects, prior to joining the staff. As Director of Finance and Administration, she oversees the administrative and financial operations of the national office. An accomplished professional with a diverse background in sales and project management, Monique previously served as an Energy Conservation Analyst with the Los Angeles Unified School District where she oversaw the utility management for sch

In [122]:
## Tag tree with pattern from nathan to tom
def tag_nathans(pt):
    return parent_cycle_up(pt.get_mother_element(), pt.nathan.parent, 'nathan', 0)

def tag_toms(pt):
    return parent_cycle_up(pt.get_mother_element(), pt.tom.parent, 'tom', 0)

def parent_cycle_up(motherElement, element, atr, num):
    ## for the bertha case we stop at the element that is right befor the mother elemenet
    if element.parent is motherElement:
        element[atr] = num
        element['sib'] = atr
    else:
        element[atr] = num
        return(motherElement, element.parent, atr, num + 1)

def reset_tree(pt):
    ## Clear all tom, nathan and sib atributes
    gm = pt.get_mother_element()
    
    for tomTag in gm.find_all(has_tom):
        del tomTag['tom']    
    for nathanTag in gm.find_all(has_nathan):
        del nathanTag['nathan']    

In [35]:
## For Each pointer
# Tag Toms and Nathans
vp = pointers[0]
tag_nathans(vp)
tag_toms(vp)

In [40]:
vp.tom.parent

<h4 class="team-member-position" sib="tom" tom="0">Senior Manager, Membership and Publications</h4>

In [41]:
## Rocket Extraction: Are you ready to launch?
vp.nathan.parent['class']

['team-member-name']

In [42]:
vp.tom.parent['class']

['team-member-position']

In [44]:
vp.get_mother_element()['class']

['team-member-right']

In [57]:
## Tom Starts
tomStarts = vh.gm.find_all(class_=vp.tom.parent['class'])

In [58]:
## Nathan Starts
nathanStarts = vh.gm.find_all(class_=vp.nathan.parent['class'])

In [35]:
## Mother Starts
motherStarts = vh.gm.find_all(class_=vp.get_mother_element()['class'])

In [58]:
vp.tom.parent.previous_sibling

<h3 class="team-member-name" nathan="0" sib="nathan">Susan Waldman</h3>

In [54]:
vp.tom.parent.previous_sibling

'Susan Waldman'

In [55]:
vp.nathan.parent.next_sibling

<h4 class="team-member-position" sib="tom" tom="0">Senior Manager, Membership and Publications</h4>

In [62]:
# Getting to a nathan with a tom start
n = 5
tomStarts[n].previous_sibling.string

'Darryl Jackson'

In [65]:
# Getting to a tom with a nathan Start
n = 0
nathanStarts[n].next_sibling.string

'President and CEO'

In [114]:
tom_rocket = lambda startNode: startNode.next_sibling.string


In [115]:
n = 7
tom_rocket(nathanStarts[n])

'Administrative Assistant'

In [116]:
nathan_rocket = lambda startNode: startNode.previous_sibling.string

In [117]:
n = 7
nathan_rocket(tomStarts[n])

"Davana O'Brien"

In [118]:
nathan_missile = lambda startNode: startNode.contents[0].string

In [119]:
n = 7
nathan_missile(motherStarts[n])

"Davana O'Brien"

In [120]:
tom_missile = lambda startNode: startNode.contents[0].next_sibling.string

In [121]:
n = 7
tom_missile(motherStarts[n])

'Administrative Assistant'

In [ ]:
## begin recursions
## We are tagged and ready

def nathan_to_tom_rocket(pt):
    if 
    
def tom_to_nathan_rocket(pt):
    
def mother_to_tom_missile(pt):
    
def mother_to_nathan_missile(pt):
    


In [123]:
vp.get_mother_element()

<div class="team-member-right"><h3 class="team-member-name" nathan="0" sib="nathan">Susan Waldman</h3><h4 class="team-member-position" sib="tom" tom="0">Senior Manager, Membership and Publications</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest. She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industry. Susan has served as the Marketing Researc

In [ ]:
#chain demo 1
def next_sib_gen(n):
    return lambda n:  

In [129]:
next_sib_gen = lambda elm, n: elm if n == 0 else next_sib_gen(elm.next_sibling, n - 1)

In [134]:
next_sib_gen(vp.get_mother_element().contents[0],2)

<div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest. She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industry. Susan has served as the Marketing Research Manager at Vibe, Spin and The Source magazines and Research Director at Motor Trend magazine. Supporting 20 sales representatives by providing them with competitive and analytic research regarding ea

In [143]:
## USELESS, but good attempt.

next_sib = lambda elm: elm.next_sibling
prev_sib = lambda elm: elm.previous_sibling
parent_up = lambda elm: elm.parent
child_down = lambda elm: elm.contents[0]
string_out = lambda elm: elm.string

In [144]:
string_out(prev_sib(tomStarts[0]))

'Eglon E. Simons'

In [146]:
string_out(prev_sib)

AttributeError: 'function' object has no attribute 'string'

In [148]:
rocketString = ".previous_sibling.string"

In [150]:
## We have a winner! The rocketString eval model
nathan_rocket_lambda = lambda elm: eval('elm' + rocketString)

In [ ]:
## This one has promise, look at it!
def left_right (arg):
    if arg == 'Left'
        return lambda elm: left_right(right)

In [166]:
## This one has promise, look at it! Pure higher order functions no strings
two_right = lambda elm: tratorx(1, elm).string

In [167]:
def tratorx(n, elm):
    if n == 0:
        return elm
    else:
        return tratorx(n-1,elm).next_sibling

In [168]:
## Another Winner!
two_right(nathanStarts[0])
## Time to fly!!

'President and CEO'

## Missile Work

In [177]:
## Helpers
def contents_position(elm):
    return contents_position_loop(elm, 0)

def contents_position_loop(elm, num):
    if len(list(elm.previous_siblings)) == 0:
        return num
    else:
        return contents_position_loop(elm.previous_sibling, num + 1)
        
def has_tom(tag):
    return 'tom' in tag.attrs

def has_nathan(tag):
    return 'nathan' in tag.attrs

def check_siblings(sibs, attFunc):
    for sib in sibs:
        if attFunc(sib):
            return True
    return False
        

In [191]:
def mother_to_tom_missile(pt):
    motherElement = pt.get_mother_element()
    
    if pt.tom is motherElement:
        return lambda start: start
    else:
        return lambda start: cycle_up_mother(pt.tom.parent, motherElement, start).contents[contents_position(pt.tom)]
    
def mother_to_nathan_missile(pt):
    motherElement = pt.get_mother_element()
    
    if pt.nathan is motherElement:
        return lambda start: start
    else:
        return lambda start: cycle_up_mother(pt.nathan.parent, motherElement, start).contents[contents_position(pt.nathan)]

def cycle_up_mother(elm, motherElement, start):
    if elm is motherElement:
        return start
    else:
        return cycle_up_mother(elm.parent, motherElement, start).contents[contents_position(elm)]
    

In [179]:
tom_missile = mother_to_tom_missile(vp)

In [180]:
tom_missile

<function __main__.mother_to_tom_missile.<locals>.<lambda>>

In [190]:
n = 7
tom_missile(motherStarts[n])

'Administrative Assistant'

In [192]:
nathan_missile = mother_to_nathan_missile(vp)

In [201]:
n = 6
nathan_missile(motherStarts[n])

'Brandon D. Hewlett'

## Rocket Time !!

In [207]:
def nathan_to_tom_rocket(pt):
    ## Compute the route from nathan startnode to tom by starting at tom and recursing up, accross and then down
    ## looing for nathan = 0 attribute
    return lambda start: cycle_up(pt.tom, 'nathan', start)
    
def tom_to_nathan_rocket(pt):
    ## Compute the route from tom startnode to nathan by starting at nathan and recursing up, accross and then down
    ## looing for tom = 0 attibute
    return lambda start: cycle_up(pt.nathan, 'tom', start)
    
def cycle_up(elm, to, start):
    ## Set Attribut Search Function
    attrFunc = has_tom if to == 'tom' else has_nathan
    
    ## First look Left for shoulder in siblings
    if check_siblings(elm.previous_siblings, attrFunc):
        return cycle_left(elm.previous_sibling, to, start).next_sibling
    ## Then look right
    elif check_siblings(elm.next_siblings, attrFunc):
        return cycle_right(elm.next_sibling, to, start).previous_sibling
    ## if nothing here go up
    else:
        return cycle_up(elm.parent, to, start).contents[contents_position(elm)]
    
def cycle_left(elm, to, start):
    attrFunc = has_tom if to == 'tom' else has_nathan
    
    ## if this is the shoulder change functions but dont move
    if attrFunc(elm):
        return cycle_down(elm, to, start)
    ## if not check the element to the left
    else:
        return cycle_left(elm.previous_sibling, to, start).next_sibling
    
def cycle_right(elm, to, start):
    attrFunc = has_tom if to == 'tom' else has_nathan
    
    ## if this is the shoulder change functions but dont move
    if attrFunc(elm):
        return cycle_down(elm, to, start)
    ## if not check the element to the left
    else:
        return cycle_right(elm.next_sibling, to, start).previous_sibling
    
def cycle_down(elm, to, start):
    return cycle_down_loop(elm[to], start)

def cycle_down_loop(dist, start):
    ## Start Node Condition
    if dist == 0:
        return start
    else: 
        return cycle_down_loop(dist - 1, start).parent
    

In [208]:
## Rocket Tests
tom_rocket = nathan_to_tom_rocket(vp)

In [217]:
n = 7

tom_rocket(nathanStarts[n])

'Administrative Assistant'

In [219]:
nathan_rocket = tom_to_nathan_rocket(vp)

In [227]:
## Success
n = 7
nathan_rocket(tomStarts[n])

"Davana O'Brien"

## Vehicle Assembley
#### nathan_to_tom_rocket
#### tom_to_nathan_rocket
#### mother_to_nathan_missile
#### mother_to_tom_missile

#### Are all recursive constructors, and are used to get to targets by adding to the 'rocketString' variable recuscursively. Once the base case is met, a single lambda is returned which combines the element and the string of opporations in the rocket string.

### Here we go, finnish strong!

In [153]:


nathan_rocket_lambda(tomStarts[0])

'Eglon E. Simons'

In [96]:
motherStarts[0].contents[0].string

'Eglon E. Simons'

In [36]:
vp.nathan.parent


<h3 class="team-member-name" nathan="0" sib="nathan">Susan Waldman</h3>

In [19]:
vp.tom.parent

<h4 class="team-member-position">Senior Manager, Membership and Publications</h4>

In [20]:
vp.mary

<div class="team-member-right"><h3 class="team-member-name">Susan Waldman</h3><h4 class="team-member-position">Senior Manager, Membership and Publications</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest. She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industry. Susan has served as the Marketing Research Manager at Vibe, Spin and The Source mag